# Data Cleaning and Exploration

In [1]:
import xpress as xp
import pandas as pd
import numpy as np

In [2]:
customer = pd.read_excel('231013_Customer_Base.xlsx')
fraud = pd.read_excel('231013_Fraud_Cases.xlsx')
transact = pd.read_excel('231013_Transactions_Input.xlsx')

In [3]:
# Merging Transaction dataset with Customer dataset by customer_id
full = transact.merge(customer, how = 'left', on='customer_id')

In [4]:
#### some code used for understanding the data
# transact.groupby('category')["category"].count().sort_values(ascending = False)
# print(full.iloc[:2])

# full[full.transac_prob.isna()].groupby('category')["category"].count().sort_values(ascending = False)
#### From this, we know that some of the transfers have type 'income'. These do not have transaction probabilities.

# full[full.transac_prob.isna()].groupby('In_or_Out')["category"].count().sort_values(ascending = False)
#### More generally, all Transfers, Income and Interest (and only these categories) are classified as paid_in. 
#### So an easy way to subset is to get rid of all paid_in.
#### Also, get rid of cash withdrawals.

In [5]:
# Remove all cash withdrawals, and all "paid_in" data from the dataset
cleaned = full[(full.category != "Cash Withdrawal") & (full.In_or_Out != "paid_in")]

# Modelling 

## Model 1: Using only day one, minimising the expected loss, decision variable is whether to investigate or not (binary), simpified constraints (only bank_from can investigate, no external investigators), international tasks completed within one day (by having multiple people work on it)

In [6]:
# Subsetting to only the first day
dayone = cleaned[cleaned.date == "2023-10-01"]
dayone = dayone.reset_index(drop = True) # reset the index, so the indexing is 0, 1, 2, ... 
# drop = False would have made the old index into a new column in the dataframe. We don't need that, so drop = True.
# Resetting the indexing is needed so that the index of the decision variable corresponds to the index of the dataframe; this is useful for subsequent analysis
dayone

,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority,home_bank,customer_prob
0,2,Online Retailer - Electronics Purchase,199.99,Online Shopping,2023-10-01,October,10298,spending,paid_out,bank_A,bank_E,0.78,0.25,3,bank_E,0.49
1,3,Utilities Payment - Internet Service,60.00,Utilities,2023-10-01,October,10298,spending,paid_out,bank_C,bank_E,0.30,0.22,1,bank_E,0.49
2,4,Rent Payment,1500.00,Housing,2023-10-01,October,14507,spending,paid_out,bank_D,bank_E,0.57,0.53,3,bank_E,0.35
3,6,Online Retailer - Electronics Purchase,299.99,Online Shopping,2023-10-01,October,14507,spending,paid_out,bank_E,bank_E,0.75,0.65,3,bank_E,0.35
4,7,Asos - Online Fashion Shopping,55.00,Shopping,2023-10-01,October,14507,spending,paid_out,bank_B,bank_E,0.37,0.31,1,bank_E,0.35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1699,2410,Credit Card Payment - American Express **** 5678,175.00,Credit Card Payment,2023-10-01,October,17647,spending,paid_out,Intrnl,bank_D,0.66,0.36,4,bank_D,0.70
1700,2412,Home Improvement - Paint and Supplies,110.75,Home Improvement,2023-10-01,October,17031,spending,paid_out,bank_E,bank_A,0.43,0.46,2,bank_A,0.58
1701,2413,Rent Payment,1200.00,Housing,2023-10-01,October,17699,spending,paid_out,bank_D,bank_A,0.64,0.38,3,bank_A,0.66
1702,2414,Credit Card Payment - Mastercard **** 6789,150.50,Credit Card Payment,2023-10-01,October,17390,spending,paid_out,Intrnl,bank_D,0.66,0.68,4,bank_D,0.45


In [22]:
# Data
teamsize = [8, 12, 10, 10, 10]
bank_names = ['bank_A', 'bank_B', 'bank_C', 'bank_D', 'bank_E']

time = [0.25, 0.5, 1, 2]
ext_cost = [40, 60, 100, 150]

# Index Sets
n_banks = 5
n_daily_cases = np.shape(dayone)[0]
Cases = range(n_daily_cases)
Banks = range(n_banks)

# Problem set-up
prob = xp.problem('fraud')

# Decision Variable
invstg = np.array([xp.var(vartype=xp.binary, name='invstg_{0}_{1}'.format(i+1, j+1))
                    for i in Cases for j in Banks], dtype=xp.npvar).reshape(n_daily_cases, n_banks)

print(invstg)

prob.addVariable(invstg)

[[invstg_1_1 invstg_1_2 invstg_1_3 invstg_1_4 invstg_1_5]
 [invstg_2_1 invstg_2_2 invstg_2_3 invstg_2_4 invstg_2_5]
 [invstg_3_1 invstg_3_2 invstg_3_3 invstg_3_4 invstg_3_5]
 ...
 [invstg_1702_1 invstg_1702_2 invstg_1702_3 invstg_1702_4 invstg_1702_5]
 [invstg_1703_1 invstg_1703_2 invstg_1703_3 invstg_1703_4 invstg_1703_5]
 [invstg_1704_1 invstg_1704_2 invstg_1704_3 invstg_1704_4 invstg_1704_5]]


In [26]:
dayone["priority"]

0       3
1       1
2       3
3       3
4       1
       ..
1699    4
1700    2
1701    3
1702    4
1703    1
Name: priority, Length: 1704, dtype: int64

### Simplified constraints: no external investigators, only bank_from can investigate

In [33]:
# Constraints

for b in Banks:
    prob.addConstraint(
        sum(invstg[i,b] * time[dayone["priority"][i]-1] for i in Cases) <= teamsize[b])

for i in Cases:
    for b in Banks:
        if dayone["bank_from"][i] != bank_names[b]:
            invstg[i,b] = 0



In [42]:
# Objective function: minimising loss of non-investigated cases

prob.setObjective(xp.Sum(dayone["transac_prob"][i] * dayone["customer_prob"][i] * dayone["Amount"][i] * (xp.Sum(invstg[i,b] for b in Banks)) for i in Cases), 
                  sense = xp.maximize)

In [43]:
prob.solve()

FICO Xpress v9.2.2, Hyper, solve started 9:29:20, Nov 16, 2023
Heap usage: 5447KB (peak 10MB, 5597KB system)
Maximizing MILP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
        10 rows         8520 cols        10224 elements      8520 entities
Presolved problem has:
         5 rows          358 cols          358 elements       358 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 8127KB (peak 10MB, 5597KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 2.50e-01,  2.00e+00] / [ 1.25e-01,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.20e+01] / [ 1.00e+00,  6.00e+00]
  Objective      [min,max] : [ 2.02e-01,  9.05e+02] / [ 1.30e+01,  9.05e+02]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 8.5GB
Fixed 273 (of 273) columns in 4 (of 4) subproblems

Starting concurrent s

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [44]:
print(f'The objective function value is {prob.getObjVal()}') 

The objective function value is 36104.670000000006


In [45]:
invstg_df = pd.DataFrame(data = prob.getSolution(invstg), index = Cases, columns = bank_names)

# This dataframe shows which bank investigated which case.

# NOTE: The indexing on this dataframe is not the transaction_id! It corresponds to the index of the original dayone df.
# Hence for extra clarity, I append a column of the actual transaction ID to the investigated cases df.
invstg_df = invstg_df.join(dayone["transaction_id"])

# Extra code for aesthetics: moving transaction_id to the left
cols = ['transaction_id'] + bank_names
invstg_df = invstg_df[cols]

# All cases which were actually investigated
invstg_df_yes = invstg_df[invstg_df[bank_names].sum(axis=1) == 1]

# # Uncomment the following code to display ALL invstg_df decision variables:
# invstg_df_disp = invstg_df.style.set_caption('Investigation Decisions').format(precision=0)
# display(invstg_df_disp)


# Uncomment the following code to display all the cases INVESTIGATED in a nicely-formatted style:
invstg_df_yes_disp = invstg_df_yes.style.set_caption('Cases Investigated').format(precision=0)
display(invstg_df_yes_disp)

,transaction_id,bank_A,bank_B,bank_C,bank_D,bank_E
14,22,0,0,0,1,0
101,149,0,1,0,0,0
123,181,0,0,1,0,0
172,250,0,0,0,0,1
259,378,0,0,0,1,0
271,396,1,0,0,0,0
287,418,0,0,1,0,0
328,474,0,0,0,1,0
341,489,0,0,0,0,1
404,578,1,0,0,0,0


### Naive model success rate, and comparison with Perfect Information case 

In [46]:
# Actual money lost from fraud: 
d1frauds = dayone.loc[dayone["transaction_id"].isin(fraud["transaction_id"])] # find all the rows in 'dayone' which are frauds by matching transaction_id

d1frauds = d1frauds.loc[~d1frauds["transaction_id"].isin(invstg_df_yes["transaction_id"])] # filter out all the rows which were investigated

# print out all the frauds that were caught:
d1frauds.loc[d1frauds["transaction_id"].isin(invstg_df_yes["transaction_id"])]
# An empty data frame is printed. I.e. NONE of the actual fraud cases were caught.

,transaction_id,description,Amount,category,date,month,customer_id,type,In_or_Out,bank_to,bank_from,transac_prob,description_prob,priority,home_bank,customer_prob


In [47]:
money_lost = d1frauds["Amount"].sum(axis=0)
money_lost

# Under perfect information, you would have investigated and found £1418.75 of scams

1418.7500000000002

## Model 2: GOALS:
### - Day one: minimising the expected loss
### - Subsequent days: minimising the actual loss of all previous days
### - Constraints: 50/50 split for all investigations involving two banks, external investigators included, international tasks completed within one day (by having multiple people work on it)
### - Decision variable: Weights on probabilities

## Currently, everything below has the correct constraints, but only minimises expected loss on day one, and has decision variable as binary. Hence, what needs to be done is (1) do the model for subsequent days, and (2) change the decision variable to be based on some updated weights on the probabilities

In [48]:
# Data
teamsize = [8, 12, 10, 10, 10]
bank_names = ['bank_A', 'bank_B', 'bank_C', 'bank_D', 'bank_E']

time = [0.25, 0.5, 1, 2]
ext_cost = [40, 60, 100, 150]

# Index Sets
n_banks = 5
n_daily_cases = np.shape(dayone)[0]
Cases = range(n_daily_cases)
Banks = range(n_banks)

# Problem set-up
prob2 = xp.problem('fraud2')

# Decision Variable


# z forces the 50:50 split
z = np.array([xp.var(vartype=xp.binary, name='z_{0}'.format(i+1))
                    for i in Cases], dtype=xp.npvar).reshape(n_daily_cases)

# no. of external investigators hired by bank b
ext = np.array([xp.var(vartype=xp.binary, name='ext_{0}_{1}'.format(i+1, b+1))
                    for i in Cases for b in Banks], dtype=xp.npvar).reshape(n_daily_cases, n_banks)



# print(invstg)

# indicator variable for when a case is shared
shared = np.array([xp.var(vartype=xp.binary, name='shared_{0}_{1}'.format(i+1, b+1))
                    for i in Cases for b in Banks], dtype=xp.npvar).reshape(n_daily_cases, n_banks)
solo = np.array([xp.var(vartype=xp.binary, name='solo_{0}_{1}'.format(i+1, b+1))
                    for i in Cases for b in Banks], dtype=xp.npvar).reshape(n_daily_cases, n_banks)

# time counter
time_taken = np.array([xp.var(vartype = xp.continuous, name='time_taken_{0}'.format(b+1))
                    for b in Banks], dtype=xp.npvar).reshape(n_banks)

# auxiliary
aux = np.array([xp.var(vartype = xp.integer, name='aux_{0}'.format(i+1))
                    for i in Cases], dtype=xp.npvar).reshape(n_daily_cases)

prob2.addVariable(z, ext, shared, solo, time_taken, aux)


In [49]:
bank_names.index("bank_A")

0

In [50]:
# Constraints
for i in Cases:
    for b in Banks:
        if (dayone["bank_from"][i] != bank_names[b]) & (dayone["bank_to"][i] != bank_names[b]):
            prob2.addConstraint(shared[i,b] == 0)
            prob2.addConstraint(solo[i,b] == 0)
            prob2.addConstraint(ext[i,b] == 0)
        elif (dayone["bank_to"][i] == "Intrnl") & (dayone["bank_from"][i] == bank_names[b]):
            prob2.addConstraint(shared[i,b] == 0)
            prob2.addConstraint(solo[i,b] == z[i])
            prob2.addConstraint(ext[i,b] <= solo[i,b])
        elif (dayone["bank_to"][i] == bank_names[b]) & (dayone["bank_from"][i] == bank_names[b]):
            prob2.addConstraint(shared[i,b] == 0)
            prob2.addConstraint(solo[i,b] == z[i])
            prob2.addConstraint(ext[i,b] <= solo[i,b])
        else:
            prob2.addConstraint(shared[i,b] == z[i]) # this forces the 50:50 split
            prob2.addConstraint(solo[i,b] == 0)
            prob2.addConstraint(ext[i,b] <= shared[i,b])

for b in Banks:
    prob2.addConstraint(
        sum(shared[i,b] * time[dayone["priority"][i]-1]/2 for i in Cases) +
         sum(solo[i,b] * time[dayone["priority"][i]-1] for i in Cases) <= 
         teamsize[b] + sum(ext[i,b] * time[dayone["priority"][i]-1] for i in Cases))


In [51]:
# Objective function: maximising gain of amount investigated

prob2.setObjective(xp.Sum(dayone["transac_prob"][i] * dayone["customer_prob"][i] * dayone["Amount"][i] * z[i] for i in Cases) - 
                   xp.Sum(ext[i,b] * ext_cost[dayone["priority"][i] - 1] for i in Cases for b in Banks), 
                   sense = xp.maximize)

prob2.write("problem2 dayone","lp")

prob2.solve()

FICO Xpress v9.2.2, Hyper, solve started 9:35:39, Nov 16, 2023
Heap usage: 15MB (peak 15MB, 5597KB system)
Maximizing MILP noname using up to 8 threads and up to 15GB memory, with these control settings:
OUTPUTLOG = 1
Original problem has:
     25565 rows        28973 cols        57166 elements     28968 entities
Presolved problem has:
      2643 rows         4246 cols        10841 elements      4246 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 18MB (peak 36MB, 5597KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.25e-01,  2.00e+00] / [ 6.25e-02,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.20e+01] / [ 1.00e+00,  2.70e+01]
  Objective      [min,max] : [ 2.02e-01,  9.05e+02] / [ 2.02e-01,  9.05e+02]
Autoscaling applied standard scaling

Symmetric problem: generators: 12, support set: 72
 Number of orbits: 36, largest orbit: 2
 Row orbits: 24, row support: 48
Will try to keep 

(<SolveStatus.COMPLETED: 3>, <SolStatus.OPTIMAL: 1>)

In [52]:
print(f'The objective function value is {prob2.getObjVal()}') 

The objective function value is 171803.1344750016


In [55]:
ext_df = pd.DataFrame(data = prob2.getSolution(ext), index = Cases, columns = bank_names)
shared_df = pd.DataFrame(data = prob2.getSolution(shared), index = Cases, columns = bank_names)
solo_df = pd.DataFrame(data = prob2.getSolution(solo), index = Cases, columns = bank_names)
z_df = pd.DataFrame(data = prob2.getSolution(z), index = Cases)

# This dataframe shows which bank SHARED investigations.

# NOTE: The indexing on this dataframe is not the transaction_id! It corresponds to the index of the original dayone df.
# Hence for extra clarity, I append a column of the actual transaction ID to the investigated cases df.
shared_df = shared_df.join(dayone["transaction_id"])

# Extra code for aesthetics: moving transaction_id to the left
cols = ['transaction_id'] + bank_names
shared_df = shared_df[cols]

# All cases which were actually investigated
shared_df_yes = shared_df[shared_df[bank_names].sum(axis=1) >= 1]

# # Uncomment the following code to display ALL decision variables:
#shared_df_disp = shared_df.style.set_caption('Investigation Decisions').format(precision=0)
#display(shared_df_disp)

# Uncomment the following code to display all the cases INVESTIGATED in a nicely-formatted style:
# shared_df_yes_disp = shared_df_yes.style.set_caption('Cases Investigated').format(precision=0)
# display(shared_df_yes_disp)

# This dataframe shows which bank did a SOLO investigation
solo_df = solo_df.join(dayone["transaction_id"])

# Extra code for aesthetics: moving transaction_id to the left
cols = ['transaction_id'] + bank_names
solo_df = solo_df[cols]

# All cases which were actually investigated
# solo_df_yes = solo_df[solo_df[bank_names].sum(axis=1) >= 1]

# # Uncomment the following code to display ALL solo decision variables:
#solo_df_disp = solo_df.style.set_caption('Investigation Decisions').format(precision=0)
#display(solo_df_disp)


# Uncomment the following code to display all the cases INVESTIGATED in a nicely-formatted style:
# solo_df_yes_disp = solo_df_yes.style.set_caption('Cases Investigated').format(precision=0)
# display(solo_df_yes_disp)

In [56]:
sum(prob2.getSolution(ext))

array([114., 113.,  99.,  96.,  89.])

## Day Two